<h1 align=center><font size = 6>Estadistica Actuarial</font></h1>

<h2>Tema: Trabajo Adicional</h2>
<h2>Autor: Jaime Gomez Marin</h2>

<p>El archivo TMSPP2017.txt se debe copiar en el directorio donde se copia este script</p>


<h3>1.-Lectura de Datos</h3>
<p>
Se lee los datos del archivo de mortandad
</p>

In [1]:
# Se importa la libreria pandas para la lectura de archivos
import pandas as pd

# Definimos el nombre del archivo de mortandad
NAMEFILE = "TMSPP2017.txt"

df = pd.read_csv(NAMEFILE, sep = "\t")


In [2]:
df.dtypes

x              int64
SPPS2017H    float64
SPPS2017M    float64
SPPI2017H    float64
SPPI2017M    float64
AaxH         float64
AaxM         float64
dtype: object

In [3]:
#print(df["SPPS2017H"])

<p> Creamos la clase Lifetable y funcion probs2lifetable() para almacenar la
    informacion de la tabla de mortandad </p>

In [4]:
class Lifetable(object):
    '''
    Definicion de clase Lifetable
    '''
    def __init__(self, x, lx, name):
        self.x = x
        self.lx = lx
        self.name = name
    

In [5]:
import numpy as np

def probs2lifetable(probs, radix=10000, type="px", name="ungiven"):
    '''
    Se crea la funcion de probabiidad probs2lifetable 
    en Python similar a R
    '''
    if(type=="px" and probs[len(probs)-1]!=0): 
        probs.append(0)
        
    if(type=="qx" and probs[len(probs)-1]!=1): 
        probs.append(1)
            
    lx = [0 for x in range(len(probs))] 
    lx[0] = radix
    
    for i in range(1,len(probs)-1) :
        if(type=="px"): 
            lx[i]=lx[i-1]*probs[i-1] 
        else: 
            lx[i]=lx[i-1]*(1-probs[i-1])
    
    return Lifetable(np.arange(1,len(probs)), lx, name) 

### Ejemplo : Cargamos las tablas de mortandad para varones sanos para el año 2019

In [6]:
probs = df["SPPS2017H"]* pow(1-df["AaxH"],2)
HS2019 = probs2lifetable(probs,radix = 10**6, type="qx",name = "HS2019")

In [7]:
# Verificamos los datos
HS2019.x

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110])

In [8]:
# Verificamos los datos
# HS2019.lx

<h3>2.-Interpretar Probabilidades</h3>
<p>
Calcular e interpretar 3 probabilidades (distintas, por ejemplo npx, nqx, m|nqx) 
</p>

Se define las funciones pxt() y qxt() en Python , tomando de referencia la implementacion del paquete lifecontigencies de R.

In [9]:
import math

# Survival probability between age x and x+t
def pxt (obj, x, t):
    
    #adjustment when age x is not an integer 
    floorx = math.floor(x) #compute floor(x)
    eps_x  = x - floorx #compute epsilon x
    u      = t + eps_x #add epsilon x to time t
    flooru = math.floor(u) #compute floor(u)
    eps_u  = u - flooru #compute epsilon t
  
    #local lifetable data closed at maximum age
    omega = max(obj.x)
    myx   = obj.x
    mylx  = obj.lx
   
    #get l_floor(x) and consecutive
    l_floorx   = mylx[floorx]
    l_floorxp1 = mylx[floorx + 1]

    #get l_floor(x+t) and consecutive
    if floorx + flooru < len(mylx):
        l_floorxu = mylx[floorx + flooru]
    else:
        l_floorxu = 0
    if floorx + flooru + 1 < len(mylx):
        l_floorxup1 = mylx[floorx + flooru + 1]
    else:
        l_floorxup1 = 0
    
    #compute one-year survival probabilites 
    if l_floorx != 0 :
        flooru_p_floorx   = l_floorxu / l_floorx 
        floorup1_p_floorx = l_floorxup1 / l_floorx
        one_p_floorx      = l_floorxp1 / l_floorx 
    else:
        flooru_p_floorx   = 0
        floorup1_p_floorx = 0
        one_p_floorx      = 0
      
    if l_floorxu != 0 :
        one_p_floorxu     = l_floorxup1 / l_floorxu
    else:
        one_p_floorxu     = 0
        
    #adjustment when t is not integer
    u_p_floorx = flooru_p_floorx * (1 - eps_u * (1 - one_p_floorxu))
  
    #adjustment when age x is not an integer (otherwise equal 1)
    eps_x_p_floorx =  1 - eps_x * (1 - one_p_floorx)

    return float(u_p_floorx / eps_x_p_floorx)


# Death probability
def qxt(obj, x, t):
    return 1 - pxt(obj=obj, x=x, t=t)


### Ejemplo 1:  $_np_x$

a.- Calcule la probabilidad de que, en el año 2020, una mujer sana de 20 años fallezca teniendo 50 años o más.

Solucion:

 $$_{30}p_{20}$$
 

In [10]:
# Cargamos la tabla de mortandad
probs = df["SPPS2017M"]* pow(1-df["AaxM"],3)
MS2020 = probs2lifetable(probs,radix = 10**6, type="qx",name = "MS2020")

# Calculamos la probabilidad
pxt(MS2020, x = 20, t = 30)

0.983113681065102

### Ejemplo 2 :  $_nq_x$ 

b.- Calcule la probabilidad de que, en el año 2018, un hombre sano de 33 años fallezca antes de cumplir 45.


Solucion:

 $$_{12}q_{33}$$

In [11]:
# Cargamos la tabla de mortandad
probs = df["SPPS2017H"]* pow(1-df["AaxH"],1)
HS2018 = probs2lifetable(probs,radix = 10**6, type="qx",name = "HS2018")

# Calculamos la probabilidad
qxt(HS2018, x = 33, t = 12)

0.014603027006928615

### Ejemplo 3 : $_{n|m}q_x$ 

c.- Calcule la probabilidad de que, en el año 2016, un hombre inválido de 60 años fallezca teniendo al menos 70 pero no más de 80 años.

Solucion:

 $$ _{10|11}q_{60} = _{10}p_{60}*_{11}q_{70} $$

In [12]:
# Cargamos la tabla de mortandad
probs = df["SPPS2017M"]* pow(1-df["AaxM"],-1)
MS2016 = probs2lifetable(probs,radix = 10**6, type="qx",name = "MS2016")

# Calculamos la probabilidad
pxt(MS2016, x = 60, t = 10)*qxt(MS2016, x = 70, t = 11)

0.15256650672064212

<h3>3.- Primas</h3>
<p>
Mostrar un ejemplo de primas (planteamiento del problema, cálculo e interpretación)
</p>

In [13]:
import math

#function to evaluate the present value of a series of cash flows
def presentValue(cashFlows, timeIds, interestRates, 
                 probabilities, power=1):
    
    v = pow(1+interestRates,-timeIds)

    #power used for APV, usually=1
    out = pow(cashFlows*v,power) * probabilities 

    return out


#function computing survival annuities
def Axn(actuarialtable, x, n = 0, i = 0, m = 0,  power = 1) :
    
    n = math.ceil((max(actuarialtable.x) + 1 - x - m) ) 
    payments = 1 
    times = m 
        
    out = 0
    for times in range(0,n-1):  
        probs = pxt(actuarialtable, x = x, t = times) \
             * qxt(actuarialtable, x = x + times, t = 1)
        out = out  \
            + presentValue(payments, timeIds=(times + 1), interestRates=i, probabilities=probs, power=power)
    
    return out 



#function computing survival annuities
def axn(actuarialtable, x, n = 0, i = 0, m = 0, power = 1) :
  
    n = math.ceil((max(actuarialtable.x) + 1 - x - m) ) 
    payments = 1 
    times = m 
    
    out = 0
    for times in range(0,n-1):  
        probs = pxt(actuarialtable, x = x, t = times)
        
        out = out  \
            + presentValue(payments, timeIds= times , interestRates=i, probabilities=probs, power=power)
    
    return out    
    

### Ejemplo 

<p> Una aseguradora emite un seguro vitalicio a un señor sano de 60 años por un monto de 200 mil soles, 
    los cuales serán pagados a sus beneficiarios al final del año en el que fallezca el asegurado. 
    Determine el monto de la prima neta a ser pagada, de manera indefinida, considerando una tasa de interés fija $i=4\%$ </p>
    
$$P=\frac{A_{60}}{\ddot{a}_{60}}$$

In [14]:
# Cargamos la tabla de mortandad
probs = df["SPPS2017H"]* pow(1-df["AaxH"],2)
HS2018 = probs2lifetable(probs,radix = 10**6, type="qx",name = "HS2018")

# Usamos las funciones creadas en Python para Axn() y axn()
A60 = Axn(HS2018, x=60, i=0.04)
a60 = axn(HS2018, x=60, i=0.04)

#
(A60/a60)*200000

4790.7655288667775

La prima neta a pagar es S/. 4790.7655